In [1]:
from langchain_community.document_loaders.text import TextLoader
from langchain_community.vectorstores.faiss import FAISS
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
def pretty_print_docs(docs):
    if isinstance(docs[0], tuple):
        print(
            f"\n{'-' * 100}\n".join(
                [
                    f"Document {i+1}: ({d[1]})\n\n{d[0].page_content}\nMetadata: {d[0].metadata}"
                    for i, d in enumerate(docs)
                ]
            )
        )
    else:
        print(
            f"\n{'-' * 100}\n".join(
                [
                    f"Document {i+1}:\n\n{d.page_content}\nMetadata: {d.metadata}"
                    for i, d in enumerate(docs)
                ]
            )
        )

K = 4

In [3]:
document_name = "nous_sommes_en_guerre.txt"

documents = TextLoader(
    document_name, encoding="utf-8"
).load()

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)
for idx, text in enumerate(texts):
    text.metadata["id"] = idx
embedding = OllamaEmbeddings(model="mistral")

In [13]:
vectordb = FAISS.from_documents(texts, embedding)
vectordb.save_local(document_name.replace(".txt", ""))

In [12]:
vectordb = FAISS.load_local(document_name.replace(".txt", ""), embeddings=embedding, allow_dangerous_deserialization=True)

In [10]:
query = "Qu'a dit le président à propos des bars et restaurants ?"

In [13]:
docs_proba = vectordb.similarity_search_with_score(query, k=K)
pretty_print_docs(docs_proba)

Document 1: (88386.0546875)

Le gouvernement, dès demain, précisera toutes ces mesures. Elles seront en fonction des besoins, des réalités économiques, des nécessités secteur par secteur, évidemment adaptées. Nous serons au rendez-vous pour que notre économie soit préservée dans cette période si dure et pour que l’ensemble des travailleuses et des travailleurs puissent avoir cette sécurité aussi en termes de pouvoir d’achat, de continuité de leur vie.
Metadata: {'source': 'nous_sommes_en_guerre.txt', 'id': 19}
----------------------------------------------------------------------------------------------------
Document 2: (107936.9375)

Dès mercredi, en conseil des ministres, sera présenté un projet de loi permettant au gouvernement de répondre à l’urgence et, lorsque nécessaire, de légiférer par ordonnance dans les domaines relevant strictement de la gestion de crise. Ce projet sera soumis au Parlement dès jeudi. J’ai vu tout à l’heure les présidents de l’Assemblée nationale et du Séna

In [50]:
docs_proba = vectordb.max_marginal_relevance_search(query, k=K)
pretty_print_docs(docs_proba)

Document 1:

Le gouvernement, dès demain, précisera toutes ces mesures. Elles seront en fonction des besoins, des réalités économiques, des nécessités secteur par secteur, évidemment adaptées. Nous serons au rendez-vous pour que notre économie soit préservée dans cette période si dure et pour que l’ensemble des travailleuses et des travailleurs puissent avoir cette sécurité aussi en termes de pouvoir d’achat, de continuité de leur vie.
Metadata: {'source': 'nous_sommes_en_guerre.txt', 'id': 19}
----------------------------------------------------------------------------------------------------
Document 2:

Mes chers compatriotes, je mesure l’impact de toutes ces décisions sur vos vies. Renoncer à voir ses proches, c’est un déchirement. Stopper ses activités quotidiennes, ses habitudes, c’est très difficile. Cela ne doit pas nous empêcher de garder le lien, d’appeler nos proches, de donner des nouvelles, d’organiser aussi les choses avec nos voisins. D’inventer de nouvelles solidarités 